In [87]:
import matplotlib.pyplot as plt
words = open('names.txt', 'r').read().splitlines()

In [16]:
# bigram_dict = {}
# for w in words:
#     chars = ['<start>'] + list(w) + ['<end>']
#     for it1, it2 in zip(chars, chars[1:]):
#         bigram = (it1, it2)
#         bigram_dict[bigram] = bigram_dict.get(bigram, 0) + 1

In [17]:
# len(bigram_dict)

In [18]:
# sorted(bigram_dict.items(), key = lambda kv: -kv[1])

In [9]:
import torch
#torch.zeros((2,3), dtype=torch.int32)

In [13]:
chars = sorted(list(set(''.join(words))))
chars.append('<start>')
chars.append('<end>')
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for i,s in enumerate(chars)}

In [14]:
count_matrix = torch.zeros((len(chars), len(chars)), dtype=torch.int32)

In [48]:
for w in words:
    ch = ['<start>'] + list(w) + ['<end>']
    for it1, it2 in zip(ch, ch[1:]):
        ix1 = stoi[it1]
        ix2 = stoi[it2]
        count_matrix[ix1, ix2] += 1

In [50]:
count_matrix[0]

tensor([ 1668,  1623,  1410,  3126,  2076,   402,   504,  6996,  4950,   525,
         1704,  7584,  4902, 16314,   189,   246,   180,  9792,  3354,  2061,
         1143,  2502,   483,   546,  6150,  1305,     0, 19920],
       dtype=torch.int32)

In [66]:
gen = torch.Generator().manual_seed(245938472738)
# p = torch.rand(3, generator = gen)
# p = p / p.sum()

In [69]:
# below it's count_matrix + 1, to smooth the model regarding 
prob_matrix = (count_matrix + 1).float()
prob_matrix /= prob_matrix.sum(1, keepdim=True)

In [70]:
for _ in range(10):
    ix = 0
    string = ""
    while True:
        p = prob_matrix[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=gen).item()
        if itos[ix] == '<start>':
            continue
        if itos[ix] == '<end>':
            break
        string += itos[ix]
    print(string)

shazx

d
lellimili

lluromry
gand
ce

hrren


In [71]:
# Goal maximize likelihood of the data wrt model parameters (statistical modeling)

# equivalent to maximizing the log likelihood (because log is monotonic)
# equivalent to minimizing the negative likelihood
# equivalent to minimizing the average negative log likelihood

# log(a*b*c) = log (a) + log (b) + log(c)

In [75]:
log_likelihood = 0.0
log_items_count = 0
for w in words[:]:
    ch = ['<start>'] + list(w) + ['<end>']
    for ch1, ch2 in zip(ch, ch[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = prob_matrix[ix1, ix2]
        log_likelihood += torch.log(prob)
        log_items_count += 1
neg_log_likelihood = -log_likelihood
avg_nll = neg_log_likelihood / log_items_count
avg_nll.item()

2.4544034004211426

In [181]:
xs = []
ys = []
for w in words:
    ch = ['<start>'] + list(w) + ['<end>']
    for ch1, ch2 in zip(ch, ch[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [182]:
import torch.nn.functional as F

# randomly initializing len(chars) no of neurons, each neuron receives len(chars) inputs
W = torch.randn((len(chars), len(chars)), requires_grad=True)

# xenc = F.one_hot(xs, num_classes=len(chars)).float()
# plt.imshow(xenc)

In [200]:
# loop for forward, backward, and updation - gradient descent
for _ in range(10):
    # forward
    xenc = F.one_hot(xs, num_classes=len(chars)).float()
    # log-counts
    logits = xenc @ W
    # exponent function - generate equivalent N
    counts = logits.exp()
    # softmax activation function
    probs = counts / counts.sum(1, keepdims=True)
    # loss function (mean of negative log likelihood)
    loss = -probs[torch.arange(len(ys)), ys].log().mean()
    print(loss.item())

    # backward
    W.grad = None
    loss.backward()

    # W updation
    W.data += -50 * W.grad

2.4596476554870605
2.459627866744995
2.459608316421509
2.4595890045166016
2.4595694541931152
2.459550619125366
2.459531307220459
2.459512710571289
2.45949387550354
2.45947527885437


In [206]:
## words generation

for _ in range(10):
    ix = 0
    string = ""
    while True:
        ## using bigram model ##
#         p = prob_matrix[ix]
        ########################
        ## using neural network ##
        xenc = F.one_hot(torch.tensor([ix]), num_classes=len(chars)).float()
        logits = xenc @ W
        counts = logits.exp()
        p = counts / counts.sum(1, keepdims=True)
        ##########################
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=gen).item()
        if itos[ix] == '<start>':
            continue
        if itos[ix] == '<end>':
            break
        string += itos[ix]
    print(string)

yn
llyllash
ayar
y
ni
m
rapae
ieletatynayndivizaerelyala
raceeee
sara
